<a href="https://colab.research.google.com/github/opasta6/opasta6/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00


In [ ]:
import os
from openai import OpenAI

os.environ["OPENAI_API_KEY"] = "自分のキー"

client = OpenAI()

In [ ]:
system_prompt="""
あなたはコンビニ店員です．レジで受付をしています．
"""

In [ ]:
def generate_response(history):
  response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
      {"role": "system", "content": system_prompt},
      *history,
    ]
  )

  return response.choices[0].message.content

In [ ]:
from pathlib import Path

speech_file_path = Path.cwd() / "speech.mp3"

def create_speech(input):
  with client.audio.speech.with_streaming_response.create(
      model="tts-1",
      voice="alloy",
      input=input
  ) as response:
      response.stream_to_file(speech_file_path)

create_speech("おはようございます")

Audio saved to: /content/speech.mp3


In [ ]:
## 生成したmp3ファイルを再生する
from IPython.display import Audio
Audio(speech_file_path, autoplay=True)

In [ ]:
!pip install pydub

In [ ]:
from IPython.display import Audio, display
from google.colab import output
from pathlib import Path
import time
from pydub import AudioSegment

speech_file_path = Path.cwd() / "speech.mp3"

# 会話記録
history = []

# 音声の長さを取得する関数
def get_audio_duration(file_path):
    audio = AudioSegment.from_mp3(file_path)
    return len(audio) / 1000.0  # ミリ秒から秒に変換

# 以下の処理をループする
while True:
    # ユーザーの入力を受け付ける
    user_input = input("ユーザー: ")

    # bと入力したらループを終了
    if user_input.lower() == "b":
        break

    # ユーザーの入力を元にLLMへの入力を作成
    history.append({"role": "user", "content": user_input})

    # ユーザーの入力を元にLLMで返答を生成
    llm_response = generate_response(history)
    history.append({"role": "assistant", "content": llm_response})
    print(f"アシスタント: {llm_response}")

    # 生成した返答を元に音声ファイルを生成
    create_speech(llm_response)

    # 音声ファイルの長さを取得
    audio_duration = get_audio_duration(speech_file_path)

    # 音声を再生
    display(Audio(speech_file_path, autoplay=True))

    # 音声の長さ分だけ待機
    time.sleep(audio_duration)

    print("音声再生が完了しました。次の入力を受け付けます。")

print("プログラムを終了します。")

ユーザー: (商品を渡し)お願いします
アシスタント: ありがとうございます。お会計をいたしますので、少々お待ちください。お支払い方法は現金ですか、クレジットカードですか？
Audio saved to: /content/speech.mp3


音声再生が完了しました。次の入力を受け付けます。
ユーザー: Paypayで
アシスタント: かしこまりました。PayPayでのお支払いですね。それでは、こちらのQRコードをお読み取りください。お支払いが完了しましたら、お知らせください。
Audio saved to: /content/speech.mp3


音声再生が完了しました。次の入力を受け付けます。
ユーザー: OKです
アシスタント: ありがとうございます。お支払いが確認できました。それでは、こちらの商品をお渡しします。何か他にご要望や質問がありましたら、お知らせください。ご利用ありがとうございました！
Audio saved to: /content/speech.mp3


音声再生が完了しました。次の入力を受け付けます。


Colab上ではシステムに近い(音声入力等)ライブラリをインストールできない．

In [ ]:
%pip install SpeechRecognition
!pip install sounddevice
!pip install scipy

ERROR: Could not find a version that satisfies the requirement python3-pyaudio (from versions: none)
ERROR: No matching distribution found for python3-pyaudio


In [ ]:
import speech_recognition as sr
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav
import io

def record_audio(duration=5, sample_rate=44100):
    print("録音を開始します...")
    audio = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1)
    sd.wait()
    print("録音が完了しました。")
    return audio, sample_rate

def transcribe_audio():
    r = sr.Recognizer()

    audio, sample_rate = record_audio()

    # NumPy配列をWAVファイルに変換
    wav_io = io.BytesIO()
    wav.write(wav_io, sample_rate, audio)
    wav_io.seek(0)

    with sr.AudioFile(wav_io) as source:
        audio_data = r.record(source)

        try:
            text = r.recognize_google(audio_data, language="ja-JP")
            print("文字起こし結果: " + text)
        except sr.UnknownValueError:
            print("音声を認識できませんでした")
        except sr.RequestError as e:
            print("Google Speech Recognition APIからの結果を取得できませんでした; {0}".format(e))

if __name__ == "__main__":
    transcribe_audio()

OSError: PortAudio library not found